In [ ]:
import numpy as np
import h5py
import scipy.io
from sklearn import metrics
import pandas as pd
import os
import tensorflow as tf
from tensorflow.keras.layers import (
    Embedding, Dense, Dropout, Activation, Flatten, Layer, Input,
    Concatenate, Reshape, Lambda, multiply, Permute, RepeatVector,
    Conv1D, MaxPooling1D, GlobalMaxPooling1D, LSTM, Bidirectional, TimeDistributed
)
from tensorflow.keras.models import Sequential, Model, load_model
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras import optimizers, regularizers, backend as K

print("Using TensorFlow backend:", tf.__version__)


### Load data (training and validation)

In [4]:
data_folder = "/Users/azami/Bioinformatics/deepsea_train/"

# Load a manageable subset
with h5py.File(data_folder + 'train.mat', 'r') as f:
    X = f['trainxdata']
    y = f['traindata']

    # ✅ Load first 10,000 samples only
    X_train = np.transpose(np.array(X[:, :, :10000]), axes=(2,0,1))  # shape (10000, 1000, 4)
    y_train = np.array(y[:, :10000]).T[:, 125:815]

print("✅ X_train:", X_train.shape)
print("✅ y_train:", y_train.shape)

✅ X_train: (10000, 1000, 4)
✅ y_train: (10000, 690)


In [5]:
validmat = scipy.io.loadmat(data_folder + 'valid.mat')

X_valid = np.transpose(validmat['validxdata'], axes=(0,2,1))[:2000]  # shape (2000, 1000, 4)
y_valid = validmat['validdata'][:2000, 125:815]  # shape (2000, 690)

print("✅ X_valid:", X_valid.shape)
print("✅ y_valid:", y_valid.shape)


✅ X_valid: (2000, 1000, 4)
✅ y_valid: (2000, 690)


### Run TBiNet

In [58]:


sequence_input = Input(shape=(1000,4))

# Convolutional Layer
output = Conv1D(320,kernel_size=26,padding="valid",activation="relu")(sequence_input)
output = MaxPooling1D(pool_size=13, strides=13)(output)
output = Dropout(0.2)(output)

#Attention Layer
attention = Dense(1)(output)
attention = Permute((2, 1))(attention)
attention = Activation('softmax')(attention)
attention = Permute((2, 1))(attention)
attention = Lambda(lambda t: tf.reduce_mean(t, axis=2), name='attention')(attention)
attention = RepeatVector(320)(attention)
attention = Permute((2,1))(attention)
output = multiply([output, attention])

#BiLSTM Layer
output = Bidirectional(LSTM(320,return_sequences=True))(output)
output = Dropout(0.5)(output)

flat_output = Flatten()(output)

#FC Layer
FC_output = Dense(695)(flat_output)
FC_output = Activation('relu')(FC_output)

#Output Layer
output = Dense(690)(FC_output)
output = Activation('sigmoid')(output)

model = Model(inputs=sequence_input, outputs=output)

print('compiling model')
model.compile(loss='binary_crossentropy', optimizer='adam')

print('model summary')
model.summary()
import os
# -------------------------------
# Prepare Folders and Callbacks
# -------------------------------
os.makedirs("./checkpoints", exist_ok=True)
os.makedirs("./model", exist_ok=True)

checkpointer = ModelCheckpoint(
    filepath="./checkpoints/tbinet.{epoch:02d}-{val_loss:.2f}.keras",
    save_best_only=False,
    verbose=1
)

earlystopper = EarlyStopping(
    monitor="val_loss",
    patience=10,
    verbose=1
)


# -------------------------------
# Train
# -------------------------------
print("Training started...")
model.fit(
    X_train,
    y_train,
    batch_size=100,
    epochs=60,
    shuffle=True,
    verbose=1,
    validation_data=(X_valid, y_valid),
    callbacks=[checkpointer, earlystopper],
)

# -------------------------------
# Save Model
# -------------------------------
save_path = "./model/tbinet.keras"
os.makedirs(os.path.dirname(save_path), exist_ok=True)
model.save(save_path)
print(f"Model saved successfully to: {save_path}")

compiling model
model summary


Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_2       │ (None, 1000, 4)   │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_3 (Conv1D)   │ (None, 975, 320)  │     33,600 │ input_layer_2[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling1d_3     │ (None, 75, 320)   │          0 │ conv1d_3[0][0]    │
│ (MaxPooling1D)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_6 (Dropout) │ (None, 75, 320)   │          0 │ max_pooling1d_3[… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_9 (Dense)     │ (None, 75, 1)     │        321 │ dropout_6[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ permute_9 (Permute) │ (None, 1, 75)     │          0 │ dense_9[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_5        │ (None, 1, 75)     │          0 │ permute_9[0][0]   │
│ (Activation)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ permute_10          │ (None, 75, 1)     │          0 │ activation_5[0][… │
│ (Permute)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ attention (Lambda)  │ (None, 75)        │          0 │ permute_10[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ repeat_vector_3     │ (None, 320, 75)   │          0 │ attention[0][0]   │
│ (RepeatVector)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ permute_11          │ (None, 75, 320)   │          0 │ repeat_vector_3[… │
│ (Permute)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ multiply_3          │ (None, 75, 320)   │          0 │ dropout_6[0][0],  │
│ (Multiply)          │                   │            │ permute_11[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bidirectional_3     │ (None, 75, 640)   │  1,640,960 │ multiply_3[0][0]  │
│ (Bidirectional)     │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_7 (Dropout) │ (None, 75, 640)   │          0 │ bidirectional_3[… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten_3 (Flatten) │ (None, 48000)     │          0 │ dropout_7[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_10 (Dense)    │ (None, 695)       │ 33,360,695 │ flatten_3[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_6        │ (None, 695)       │          0 │ dense_10[0][0]    │
│ (Activation)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_11 (Dense)    │ (None, 690)       │    480,240 │ activation_6[0][… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_7        │ (None, 690)       │          0 │ dense_11[0][0]    │
│ (Activation)        │                   │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 35,515,816 (135.48 MB)

 Trainable params: 35,515,816 (135.48 MB)

 Non-trainable params: 0 (0.00 B)

🚀 Training started...
Epoch 1/60
100/100 ━━━━━━━━━━━━━━━━━━━━ 0s 511ms/step - loss: 0.1945
Epoch 1: saving model to ./checkpoints/tbinet.01-0.05.keras
100/100 ━━━━━━━━━━━━━━━━━━━━ 58s 552ms/step - loss: 0.1935 - val_loss: 0.0547
Epoch 2/60
100/100 ━━━━━━━━━━━━━━━━━━━━ 0s 487ms/step - loss: 0.0619
Epoch 2: saving model to ./checkpoints/tbinet.02-0.05.keras
100/100 ━━━━━━━━━━━━━━━━━━━━ 53s 525ms/step - loss: 0.0619 - val_loss: 0.0544
Epoch 3/60
100/100 ━━━━━━━━━━━━━━━━━━━━ 0s 496ms/step - loss: 0.0593
Epoch 3: saving model to ./checkpoints/tbinet.03-0.05.keras
100/100 ━━━━━━━━━━━━━━━━━━━━ 53s 534ms/step - loss: 0.0593 - val_loss: 0.0547
Epoch 4/60
100/100 ━━━━━━━━━━━━━━━━━━━━ 0s 482ms/step - loss: 0.0591
Epoch 4: saving model to ./checkpoints/tbinet.04-0.05.keras
100/100 ━━━━━━━━━━━━━━━━━━━━ 52s 521ms/step - loss: 0.0591 - val_loss: 0.0534
Epoch 5/60
100/100 ━━━━━━━━━━━━━━━━━━━━ 0s 495ms/step - loss: 0.0591
Epoch 5: saving model to ./checkpoints/tbinet.05-0.05.keras
100/100 ━━━━━━━━━━━━━

## 1️⃣ Rebuild your model with the corrected Lambda (output_shape=(75,))

In [6]:
sequence_input = Input(shape=(1000,4))

output = Conv1D(320, kernel_size=26, padding="valid", activation="relu")(sequence_input)
output = MaxPooling1D(pool_size=13, strides=13)(output)
output = Dropout(0.2)(output)

attention = Dense(1)(output)
attention = Permute((2, 1))(attention)
attention = Activation('softmax')(attention)
attention = Permute((2, 1))(attention)
attention = Lambda(lambda t: tf.reduce_mean(t, axis=2),
                   output_shape=(75,), name='attention')(attention)  # ✅ fixed
attention = RepeatVector(320)(attention)
attention = Permute((2, 1))(attention)
output = multiply([output, attention])

output = Bidirectional(LSTM(320, return_sequences=True))(output)
output = Dropout(0.5)(output)
flat_output = Flatten()(output)

FC_output = Dense(695, activation='relu')(flat_output)
output = Dense(690, activation='sigmoid')(FC_output)

model = Model(inputs=sequence_input, outputs=output)
model.compile(loss='binary_crossentropy', optimizer='adam')

## Load the most recent checkpoint weights

In [7]:
import glob
latest_checkpoint = sorted(glob.glob("./checkpoints/tbinet.*.keras"))[-1]
print("Loading weights from:", latest_checkpoint)

model.load_weights(latest_checkpoint)
print("Weights loaded successfully!")


Loading weights from: ./checkpoints/tbinet.24-0.05.keras
✅ Weights loaded successfully!


/opt/homebrew/Cellar/jupyterlab/4.2.1/libexec/lib/python3.12/site-packages/keras/src/saving/saving_lib.py:576: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 30 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [8]:
data_folder = "/Users/azami/Bioinformatics/deepsea_train/"
testmat = scipy.io.loadmat(data_folder + 'test.mat')
X_test = np.transpose(testmat['testxdata'], axes=(0,2,1))
y_test = testmat['testdata'][:,125:815]

test_loss = model.evaluate(X_test, y_test, verbose=1)
print("Test loss:", test_loss)


14220/14220 ━━━━━━━━━━━━━━━━━━━━ 1811s 127ms/step - loss: 0.0506
Test loss: 0.052174970507621765


## Load test Data

In [9]:
data_folder = "/Users/azami/Bioinformatics/deepsea_train/"
testmat = scipy.io.loadmat(data_folder + 'test.mat')

## Create AUROC & AUPR function

In [10]:
def get_auroc(preds, obs):
    fpr, tpr, thresholds  = metrics.roc_curve(obs, preds, drop_intermediate=False)
    auroc = metrics.auc(fpr,tpr)
    return auroc

def get_aupr(preds, obs):
    precision, recall, thresholds  = metrics.precision_recall_curve(obs, preds)
    aupr = metrics.auc(recall,precision)
    return aupr

def get_aurocs_and_auprs(tpreds, tobs):
    tpreds_df = pd.DataFrame(tpreds)
    tobs_df = pd.DataFrame(tobs)
    
    task_list = []
    auroc_list = []
    aupr_list = []
    for task in tpreds_df:
        pred = tpreds_df[task]
        obs = tobs_df[task]
        auroc=round(get_auroc(pred,obs),5)
        aupr = round(get_aupr(pred,obs),5)
        task_list.append(task)
        auroc_list.append(auroc)
        aupr_list.append(aupr)
    return auroc_list, aupr_list

In [ ]:
tpreds = model.predict(np.transpose(testmat['testxdata'],axes=(0,2,1)),verbose=1)
tpreds_temp = np.copy(tpreds)
reverse_start_id = int(testmat['testdata'][:,125:815].shape[0]/2)

for i in range(reverse_start_id):
    tpreds_avg_temp = (tpreds_temp[i] + tpreds_temp[reverse_start_id+i])/2.0
    tpreds_temp[i] = tpreds_avg_temp
    tpreds_temp[reverse_start_id+i] = tpreds_avg_temp


aurocs, auprs = get_aurocs_and_auprs(tpreds_temp,testmat['testdata'][:,125:815])
print("Averaged AUROC:",np.nanmean(aurocs))
print("Averaged AUPR:", np.nanmean(auprs))